<a href="https://colab.research.google.com/github/AlexandreCandidoXavier/BR-DWGD/blob/main/notebooks/exemplo3_google_earth_export_geoTiff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exportando imagens BR-DWGD mensais e anual

Objetivo: exportar em "Geotiff" os dados de precipitação da grade BR-DWGD, de uma região de interesse. Considerando as escalas de tempo mensal e anual.

In [1]:
# instalando geemap atualizado (https://geemap.org/):
# !pip install -U geemap

In [2]:
import ee
import geemap
import numpy as np
import datetime
import pandas as pd

In [3]:
geemap.__version__

'0.32.0'

In [4]:
# ee.Authenticate()

In [5]:
ee.Initialize()

In [6]:
m = geemap.Map()

## Definindo região de interesse (roi)

Será desenhado a áree de interesse e com o método *.draw_features*.

In [7]:
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

Depois de desenhar, ver coordenadas.

In [8]:
# roi = ee.FeatureCollection(m.draw_features)
# roi.getInfo()

Para deixar elas salvas.

In [9]:
# Create a geodesic polygon.
# https://developers.google.com/earth-engine/guides/geometry_visualization_info
polygon = ee.Geometry.Polygon([
  [[-46.757813, -23.337212],
      [-46.757813, -1.158979],
      [-33.398438, -1.158979],
      [-33.398438, -23.337212],
      [-46.757813, -23.337212]],
]);
roi = ee.Geometry(polygon, None, False)
# roi = ee.FeatureCollection(polygon, None, False)

Se quiser gerar *buffer*

In [10]:
# Buffer
# roi_buffer = ee.Feature(roi.geometry()).buffer(20000)

In [11]:
m.addLayer(roi, {}, 'ROI')

Pegando variável precipitação da base BR-DWGD e reescalonando.
Para as demais: variáveis: https://gee-community-catalog.org/projects/br_dwgd/

In [12]:
# definir período. Definir o começo e final (start_date, end_date) , sempre no inicio e final de ano
start_date, end_date = '2016-01-01','2023-01-01'

var = ee.ImageCollection("projects/sat-io/open-datasets/BR-DWGD/PR") \
                         .filterDate(start_date, end_date)
# var = ee.ImageCollection("projects/sat-io/open-datasets/BR-DWGD/PR").filterDate('2020-12-10', '2020-12-30')
# print(var.aggregate_array('system:id').getInfo())

Reescalonar e pegando apenas a área de interesse

In [13]:
offset = 225
scale  = 0.006866665

# reescalonando
def scaleBand(image):
    scaledImage = image.select("b1").rename(['pr']) \
                       .multiply(scale).add(offset) \
                       .set({'date': image.date().format('yyyy-MM-dd')}) \
                       .clip(roi)
    return scaledImage.copyProperties(image, image.propertyNames())

dados = var.map(scaleBand)

In [14]:
dados.first()

In [15]:
# informações para plotar o mapa da precipitação
color_pal = ['000066', '001199', '0044BB', '0077DD', '33AAEE', '66CCFF', 'FFDDCC', 'FFBB99', 'FF9966', 'FF6644']
vis = {'min': 0,'max': 10, 'palette': color_pal}


m.addLayer(dados.first(), vis, 'Precipitação')

Gerando lista das datas das imagens para o período específicado. Vai ser utilizados para determinar os totais de meses e anos no período.

In [16]:
lista_datas = ee.List([])
 # Image date collection applied to each image.
def formatDate(image, lista_datas):
    data_dia = ee.List(ee.String(image.date().format('YYYY-MM-dd')))
    return ee.List(lista_datas).add(data_dia)


datas = dados.iterate(formatDate, lista_datas)
datas = datas.getInfo()

In [17]:
# datas

## Exportando arquivo *geotiff* na escala de tempo mensal

Definindo número de meses para o período de estudo

In [18]:
dias = pd.to_datetime(datas)
dia_inicial, dia_final = dias[0], dias[-1]
meses = pd.date_range(dia_inicial, dia_final, freq='MS').strftime('%Y-%m').tolist()
n_meses = len(meses) 
dia_final.strftime('%Y-%m-%d')

'2022-12-31'

In [19]:
n_meses

84

Calculando o acumulado mensal no período

In [20]:
def func_soma_mensal(n):
    start = ee.Date(dia_inicial.strftime('%Y-%m-%d')).advance(n, 'month')
    end = start.advance(1, 'month')
    
    return dados.filterDate(start, end).sum().clip(roi) 


prec_mensal = ee.List.sequence(0, n_meses - 1).map(func_soma_mensal)
prec_mensal

In [21]:
# start = ee.Date(dia_inicial.strftime('%Y-%m-%d')).advance(83, 'month')
# end = start.advance(1, 'month')
# print(start.format('YYYY-MM-dd').getInfo(), end.format('YYYY-MM-dd').getInfo())

Transformando prec_mensal para uma imagem com várias bandas

In [22]:
prec_mensal = ee.ImageCollection(prec_mensal).toBands()
# Renomeando as bandas
prec_mensal = prec_mensal.rename(meses)
prec_mensal
# prec_mensal.bandNames().getInfo()

In [23]:
m.addLayer(prec_mensal, {'bands': ['2022-01'], 'min': 0,'max': 400, 'palette': color_pal}, 'Pr_mensal')

In [24]:
roi

ee.Geometry({
  "functionInvocationValue": {
    "functionName": "GeometryConstructors.Polygon",
    "arguments": {
      "coordinates": {
        "constantValue": [
          [
            [
              -46.757813,
              -23.337212
            ],
            [
              -46.757813,
              -1.158979
            ],
            [
              -33.398438,
              -1.158979
            ],
            [
              -33.398438,
              -23.337212
            ],
            [
              -46.757813,
              -23.337212
            ]
          ]
        ]
      },
      "evenOdd": {
        "constantValue": true
      },
      "geodesic": {
        "constantValue": false
      }
    }
  }
})

Gravando

In [25]:
geemap.ee_export_image(prec_mensal,
                       scale=11000,
                       region=roi,
                       file_per_band=False,
                       filename='dados_roi_mensal.tif')

Generating URL ...
Please wait ...
Data downloaded to /home/alexandre/Dropbox/grade_2020/examples/BR-DWGD/notebooks/dados_roi_mensal.tif


## Exportando arquivo *geotiff* na escala de tempo anual

Definindo número de anos para o período de estudo

In [26]:
anos = pd.date_range(dia_inicial, dia_final, freq='Y').strftime('%Y').tolist()
n_anos = len(anos)
n_anos

7

In [27]:
def func_media_anual(n):
    start = ee.Date(dia_inicial.strftime('%Y-%m-%d')).advance(n, 'year')
    end = start.advance(1, 'year')
    
    return dados.filterDate(start, end).sum().clip(roi) 


prec_anual = ee.List.sequence(0, n_anos - 1).map(func_media_anual)
prec_anual

Transformando **lista** de bandas da prec_anual para uma imagem única com várias bandas

In [28]:
prec_anual = ee.ImageCollection(prec_anual).toBands()
prec_anual = prec_anual.rename(anos)
prec_anual
# prec_mensal.bandNames().getInfo()

In [29]:
m.addLayer(prec_anual, {'bands': ['2022'], 'min': 0,'max': 2000, 'palette': color_pal}, 'Pr_anual')

In [30]:
geemap.ee_export_image(prec_anual,
                       scale=11000,
                       region=roi,
                       # file_per_band=True,
                       filename='dados_roi_anual.tif')

Generating URL ...
Please wait ...
Data downloaded to /home/alexandre/Dropbox/grade_2020/examples/BR-DWGD/notebooks/dados_roi_anual.tif
